---
title: p5.js教程07-让图动起来
date: 2018-06-06 20:17:55
tags: [p5.js]
toc: true
xiongzhang: true


---
<span></span>
<!-- more -->

> 声明: 本文由[DataScience](http://mlln.cn)发表,未经允许不得转载。 转载请注明[本文链接](http://mlln.cn)mlln.cn, 并在文后留言`转载`.

本文代码运行环境:

- windows10
- jupyter notebook
- p5.js 0.6.1

在此之前的教程中你可能已经见过动态的效果了, 不过我们在这里更详细和系统的说明下。

就像翻书一样，屏幕上的动画是通过不断显示不同的图像来显示动态效果的。我们需要绘制图像，然后绘制稍微不同的图像，然后绘制另一个图像，依此类推, 让这些图像连续出现。当以足够快的速度呈现一组相似图像时，我们的大脑将这些图像转换为运动。

(下面我们注册p5魔法, 以便于在jupyter notebook中使用p5.js, 如果你不用notebook, 请忽略这部分python代码)

In [46]:
from IPython.core.magic import register_cell_magic
from IPython.display import IFrame

TEMPLATE = """
<html>
<head>
<meta charset="utf-8" />
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.1/p5.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.1/addons/p5.dom.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/p5.js/0.6.1/addons/p5.sound.min.js"></script>
<title>Notebook中显示P5.js页面</title>
</head>
<body>
<a href="http://mlln.cn"  target="_blank"><h5>p5.js效果展示: %(name)s</h5></a>
<script>
%(script)s
</script>
</body>
</html>
"""


@register_cell_magic
def p5(line, cell):
    file_id, kws = line.split(' ')[0], line.split(' ')[1:]
    kwargs = {}
    for kw in kws:
        k, v = kw.split('=')
        kwargs[k] = v
    filename = f"p5js-html/p5-07-{file_id}.html"
    with open(filename, "w", encoding='utf8') as fp:
        fp.write(TEMPLATE % {"script": cell, "name": filename})
    if 'height' not in kwargs:
        kwargs['height'] = '200px'
    return IFrame(filename, width="100%", **kwargs)

del p5

### 帧的概念

为了创建平滑的运动，p5.js尝试以每秒60帧的速度运行draw（）内的代码。帧是draw（）函数的一次执行，帧速率是每秒绘制的帧数。因此，每秒绘制60帧的程序意味着程序每秒60运行60次draw（）函数。

### 控制运动速度和方向

通常我们用物体在每帧的位移大小来表示速度。

In [47]:
%%p5 speed height=250
// 速度是每帧1个像素
let speed = 1
// 方向有1 和 -1两个取值
let direction = 1
let x = 1
function setup() {  
    createCanvas(220, 220);  
    
}

function draw() {
    // 更改小球的x坐标实现运动效果
    x += speed*direction;
    background(204); 
    ellipse(x, 100, 20, 20); 
    if(x > 220){
        direction = -1
    }else if(x < 0){
        direction = 1
    }
}

更复杂的案例, 制作一个随机运动的小球。

In [48]:
%%p5 speed-random height=280
// 速度是每帧1个像素
// 用random生成一个随机数
let speed_x = 1 
let speed_y = 2 
// 方向有1 和 -1两个取值
let direction_x = 1
let direction_y = 1
let x = 1
let y = 50
function setup() {  
    createCanvas(220, 220);  
    // 使用random方法可以生成处于两个参数之间的数字
    speed_x *= random(1, 10) * 0.1
    speed_y *= random(5, 10) * 0.1
    // 方向也是随机的
    if (random(0, 1)>0.5){
        direction_x *= -1
    }
    if (random(0, 1)>0.5){
        direction_y *= -1
    }
}

function draw() {
    // 更改小球的x坐标实现运动效果
    x += speed_x*direction_x;
    y += speed_y*direction_y;
    background(204); 
    ellipse(x, y, 20, 20); 
    // 判断小球在x方向上是否出界
    if(x > 210){
        direction_x = -1
        // 如果出界, 调转方向
    }else if(x < 10){
        direction_x = 1
    }
    // 判断小球在y方向上是否出界
    if(y > 210){
        // 如果出界, 调转方向
        direction_y = -1
    }else if(y < 10){
        direction_y = 1
    }
}

### 计时


每个p5.js程序都会计算自启动以来经过的时间。它以毫秒（千分之一秒）计，因此在1秒后，计数器为1,000; 5秒后，它达到5,000; 1分钟后，它是60,000。我们可以使用此计数器在特定时间触发动画。 millis（）函数返回此计数器值。

下面我们制作一个可以定时改变颜色的小球。

In [49]:
%%p5 speed-random-change-color height=280
// 速度是每帧1个像素
// 用random生成一个随机数
let speed_x = 1 
let speed_y = 2 
// 方向有1 和 -1两个取值
let direction_x = 1
let direction_y = 1
let x = 1
let y = 50

// 记录上次更改颜色的时间
let change_time = 0;
function setup() {  
    createCanvas(220, 220);  
    // 使用random方法可以生成处于两个参数之间的数字
    speed_x *= random(1, 10) * 0.1
    speed_y *= random(5, 10) * 0.1
    // 方向也是随机的
    if (random(0, 1)>0.5){
        direction_x *= -1
    }
    if (random(0, 1)>0.5){
        direction_y *= -1
    }
}

function draw() {
    // 更改小球的x坐标实现运动效果
    x += speed_x*direction_x;
    y += speed_y*direction_y;
    background(204); 
    ellipse(x, y, 20, 20); 
    // 判断小球在x方向上是否出界
    if(x > 210){
        direction_x = -1
        // 如果出界, 调转方向
    }else if(x < 10){
        direction_x = 1
    }
    // 判断小球在y方向上是否出界
    if(y > 210){
        // 如果出界, 调转方向
        direction_y = -1
    }else if(y < 10){
        direction_y = 1
    }
    // 获取当前时间
    let current = millis()

    // 每隔1000毫秒执行一次
    // 判断时间是否超过1000毫秒
    if (current - change_time > 1000){
        fill(random(1, 255), random(1, 255), random(1, 255))
        // change_time的值
        change_time = current
    }
}